Import necessary packages: pandas, numpy and plotly

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go

设置plotly显示结果的模式

In [2]:
plotly.offline.init_notebook_mode(connected=True)

读取csv数据到pandas DataFrame

In [3]:
online_fname = 'https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv'
local_fname = 'time_series_2019-ncov-Confirmed.csv'
# Source: https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases
df = pd.read_csv(local_fname)

用shape和head()来查看数据的基本情况

In [4]:
print(df.shape)
df.head()

(501, 66)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,82,114,147,177,212,272,322,411,599,599.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,212,226,243,266,313,345,385,432,455,455.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0


找出来占了多行的国家

In [5]:
countries, counts = np.unique(df['Country/Region'].values, return_counts = True)
multi_region_countries = {countries[i] : counts[i] for i in range(len(countries)) if counts[i] > 1}
multi_region_countries

{'Australia': 9,
 'Canada': 12,
 'China': 33,
 'Denmark': 3,
 'France': 9,
 'Netherlands': 4,
 'US': 249,
 'United Kingdom': 7}

查看每个国家的情况，都是什么样的行政单位占了很多行

In [6]:
df[df["Country/Region"] == "Denmark"]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
170,Denmark,Denmark,56.2639,9.5018,0,0,0,0,0,0,...,827,864,914,977,1057,1151,1255,1326,1395,1395.0
197,Faroe Islands,Denmark,61.8926,-6.9118,0,0,0,0,0,0,...,9,11,18,47,58,72,80,92,115,115.0
461,Greenland,Denmark,71.7069,-42.6043,0,0,0,0,0,0,...,0,0,1,1,1,2,2,2,4,4.0


其中美国比较特殊，除了州以外，还包括了county的信息

In [7]:
df[df['Country/Region'] == 'US']['Province/State'].values

array(['Washington', 'New York', 'California', 'Massachusetts',
       'Diamond Princess', 'Grand Princess', 'Georgia', 'Colorado',
       'Florida', 'New Jersey', 'Oregon', 'Texas', 'Illinois',
       'Pennsylvania', 'Iowa', 'Maryland', 'North Carolina',
       'South Carolina', 'Tennessee', 'Virginia', 'Arizona', 'Indiana',
       'Kentucky', 'District of Columbia', 'Nevada', 'New Hampshire',
       'Minnesota', 'Nebraska', 'Ohio', 'Rhode Island', 'Wisconsin',
       'Connecticut', 'Hawaii', 'Oklahoma', 'Utah', 'Kansas', 'Louisiana',
       'Missouri', 'Vermont', 'Alaska', 'Arkansas', 'Delaware', 'Idaho',
       'Maine', 'Michigan', 'Mississippi', 'Montana', 'New Mexico',
       'North Dakota', 'South Dakota', 'West Virginia', 'Wyoming',
       'Kitsap, WA', 'Solano, CA', 'Santa Cruz, CA', 'Napa, CA',
       'Ventura, CA', 'Worcester, MA', 'Gwinnett, GA', 'DeKalb, GA',
       'Floyd, GA', 'Fayette, GA', 'Gregg, TX', 'Monmouth, NJ',
       'Burlington, NJ', 'Camden, NJ', 'Passaic, NJ'

观察发现只有美国比较难搞，其它国家都可以把不同的行政区直接加起来。。。美国的包含一大堆county的名字，在Province/State中含有逗号, 通过这个来识别并去掉
注意，英国、法国等包含一个与本国名相同的Province/State, 指的是本土
所以:
1. 先drop掉美国的这些county
2. 对于其它国家，数量上把整个国家的加起来，Lat/Long按照本土的或者首都位置来设置

In [8]:
us_rows = df[df["Country/Region"] == "US"]
us_counties = us_rows[us_rows["Province/State"].apply(lambda x: ',' in x)]
df.drop(us_counties.index, inplace = True)
df.reset_index(inplace=True)

用这个函数来aggregate每一个国家的不同province/state

In [9]:
def aggregate_nation_data(country, df):
    if country == 'China':
        states_data = df[(df["Country/Region"] == 'China').values | (df["Country/Region"] == 'Taiwan*').values]
    else:
        states_data = df[df["Country/Region"] == country]
    nation_data = states_data.sum()
    nation_data['Province/State'] = np.nan
    nation_data['Country/Region'] = country
    if country in ['China', 'Australia', 'Canada']:
        nation_data['Lat'] = states_data['Lat'].values.mean()
        nation_data['Long'] = states_data['Long'].values.mean()
    else:
        nation_data['Lat'] = states_data[states_data["Province/State"] == country]['Lat'].values[0]
        nation_data['Long'] = states_data[states_data["Province/State"] == country]['Long'].values[0]
    nation_data = pd.DataFrame(nation_data)
    df.drop(states_data.index, inplace = True)
    return pd.DataFrame(nation_data)

In [10]:
countries = ['Australia', 'China', 'Canada', 'US', 'United Kingdom', 'France', 'Denmark', 'Netherlands']
for country in countries:
    aggregated = aggregate_nation_data(country, df)
    df = pd.concat([df, aggregated.T], axis = 0)
df.reset_index(inplace=True)
df.drop(['level_0', 'index', 'Province/State'], axis = 1, inplace=True)

In [11]:
df

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,Thailand,15,101,2,3,5,7,8,8,14,...,82,114,147,177,212,272,322,411,599,599
1,Japan,36,138,2,1,2,2,4,4,7,...,773,839,825,878,889,924,963,1007,1086,1086
2,Singapore,1.2833,103.833,0,1,3,3,4,5,7,...,212,226,243,266,313,345,385,432,455,455
3,Nepal,28.1667,84.25,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2
4,Malaysia,2.5,112.5,0,0,0,3,4,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,US,37.0902,-95.7129,0,0,0,0,0,0,0,...,2727,3499,4632,6421,7783,13677,19100,25489,33272,33276
178,United Kingdom,55.3781,-3.436,0,0,0,0,0,0,0,...,1144,1145,1551,1960,2642,2716,4014,5067,5741,5741
179,France,46.2276,2.2137,0,0,2,3,3,3,4,...,4487,4523,6668,7699,9105,10947,12726,14431,16176,16044
180,Denmark,56.2639,9.5018,0,0,0,0,0,0,0,...,836,875,933,1025,1116,1225,1337,1420,1514,1514


读入国家代码数据， 用于plotly package

In [12]:
country_code = pd.read_excel("country_code.xlsx")
country_code.set_index("COUNTRY")

,CODE
COUNTRY,
Afghanistan,AFG
Albania,ALB
Algeria,DZA
American Samoa,ASM
Andorra,AND
...,...
Virgin Islands,VGB
West Bank,WBG
Yemen,YEM


把两个数据集中的国家名字拿出来

In [13]:
countries_with_data = df["Country/Region"].values.tolist()
coded_countries = country_code["COUNTRY"].values.tolist()
coded_countries

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'The Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo (Kinshasa)',
 'Republic of the Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Falkland Islands (Islas Malvinas)',
 'Faroe Islands',
 'Fiji

获取差集

In [14]:
no_data_countries = [c for c in coded_countries if not c in countries_with_data]

In [15]:
no_data_countries

['American Samoa',
 'Anguilla',
 'Aruba',
 'Belize',
 'Bermuda',
 'Botswana',
 'British Virgin Islands',
 'Burma',
 'Burundi',
 'Cayman Islands',
 'Comoros',
 'Cook Islands',
 'Curacao',
 'Falkland Islands (Islas Malvinas)',
 'Faroe Islands',
 'French Polynesia',
 'Gibraltar',
 'Guinea-Bissau',
 'Hong Kong',
 'Isle of Man',
 'Kiribati',
 'Korea, North',
 'Laos',
 'Lesotho',
 'Libya',
 'Macau',
 'Macedonia',
 'Malawi',
 'Mali',
 'Marshall Islands',
 'Micronesia, Federated States of',
 'New Caledonia',
 'Niue',
 'Northern Mariana Islands',
 'Palau',
 'Saint Kitts and Nevis',
 'Saint Martin',
 'Saint Pierre and Miquelon',
 'Samoa',
 'Sao Tome and Principe',
 'Sierra Leone',
 'Sint Maarten',
 'Solomon Islands',
 'South Sudan',
 'Swaziland',
 'Taiwan',
 'Tajikistan',
 'Tonga',
 'Turkmenistan',
 'Tuvalu',
 'Vanuatu',
 'Virgin Islands',
 'West Bank',
 'Yemen']

为这些在df中没有数据的国家，创建空白数据

In [16]:
blank_countries = pd.DataFrame(np.zeros((len(no_data_countries), df.shape[1])), columns = df.columns)
blank_countries["Country/Region"] = no_data_countries

In [17]:
blank_countries

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,American Samoa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Anguilla,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Belize,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bermuda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Botswana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,British Virgin Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Burma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Burundi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Cayman Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


把在在country code中有，但是没有COVID19统计数据的国家的空白数据添加到df中

In [18]:
appended_df = pd.concat([df, blank_countries], axis = 0)

In [19]:
appended_df

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,Thailand,15,101,2,3,5,7,8,8,14,...,82,114,147,177,212,272,322,411,599,599
1,Japan,36,138,2,1,2,2,4,4,7,...,773,839,825,878,889,924,963,1007,1086,1086
2,Singapore,1.2833,103.833,0,1,3,3,4,5,7,...,212,226,243,266,313,345,385,432,455,455
3,Nepal,28.1667,84.25,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2
4,Malaysia,2.5,112.5,0,0,0,3,4,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,Tuvalu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,Vanuatu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,Virgin Islands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,West Bank,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


把空白数据拼接到df中去

In [20]:
merged_df = pd.merge(appended_df, country_code, left_on='Country/Region', right_on = 'COUNTRY', how = 'left')

In [21]:
merged_df

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,COUNTRY,CODE
0,Thailand,15,101,2,3,5,7,8,8,14,...,147,177,212,272,322,411,599,599,Thailand,THA
1,Japan,36,138,2,1,2,2,4,4,7,...,825,878,889,924,963,1007,1086,1086,Japan,JPN
2,Singapore,1.2833,103.833,0,1,3,3,4,5,7,...,243,266,313,345,385,432,455,455,Singapore,SGP
3,Nepal,28.1667,84.25,0,0,0,1,1,1,1,...,1,1,1,1,1,1,2,2,Nepal,NPL
4,Malaysia,2.5,112.5,0,0,0,3,4,4,4,...,566,673,790,900,1030,1183,1306,1306,Malaysia,MYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,Tuvalu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Tuvalu,TUV
232,Vanuatu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Vanuatu,VUT
233,Virgin Islands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Virgin Islands,VGB
234,West Bank,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,West Bank,WBG


剔除没有country code的国家

In [22]:
merged_df[merged_df.COUNTRY.isnull()]

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,COUNTRY,CODE
32,North Macedonia,41.6086,21.7453,0,0,0,0,0,0,0,...,18,26,35,48,67,85,114,114,NaN,NaN
78,Martinique,14.6415,-61.0242,0,0,0,0,0,0,0,...,15,16,19,23,32,32,37,37,NaN,NaN
87,Holy See,41.9029,12.4534,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,NaN,NaN
92,Cruise Ship,35.4437,139.638,0,0,0,0,0,0,0,...,696,696,712,712,712,712,712,712,NaN,NaN
117,Eswatini,-26.5225,31.4659,0,0,0,0,0,0,0,...,1,1,1,1,1,1,4,4,NaN,NaN
127,Congo (Brazzaville),-4.0383,21.7587,0,0,0,0,0,0,0,...,1,1,1,3,3,3,3,3,NaN,NaN
140,"Gambia, The",13.4432,-15.3101,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,NaN,NaN
141,"Bahamas, The",25.0343,-77.3963,0,0,0,0,0,0,0,...,1,1,1,3,3,4,4,4,NaN,NaN
153,Cape Verde,15.1111,-23.6167,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,NaN,NaN
154,East Timor,-8.55,125.56,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,NaN,NaN


In [23]:
cleaned_df = merged_df[merged_df.COUNTRY.notnull()]

In [24]:
cleaned_df

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,COUNTRY,CODE
0,Thailand,15,101,2,3,5,7,8,8,14,...,147,177,212,272,322,411,599,599,Thailand,THA
1,Japan,36,138,2,1,2,2,4,4,7,...,825,878,889,924,963,1007,1086,1086,Japan,JPN
2,Singapore,1.2833,103.833,0,1,3,3,4,5,7,...,243,266,313,345,385,432,455,455,Singapore,SGP
3,Nepal,28.1667,84.25,0,0,0,1,1,1,1,...,1,1,1,1,1,1,2,2,Nepal,NPL
4,Malaysia,2.5,112.5,0,0,0,3,4,4,4,...,566,673,790,900,1030,1183,1306,1306,Malaysia,MYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,Tuvalu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Tuvalu,TUV
232,Vanuatu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Vanuatu,VUT
233,Virgin Islands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Virgin Islands,VGB
234,West Bank,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,West Bank,WBG


In [25]:
def draw_figure(df, date):
    fig = go.Figure(data=go.Choropleth(
        locations = df['CODE'],
        z = df[date],
        text = df['Country/Region'],
        colorscale = 'Reds',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='lightgray',
        marker_line_width=0.5,
        colorbar_title = 'Confirmed cases',
    ))

    fig.update_layout(
        title_text='COVID19 Accumulative Confirmed Cases',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
    )
    return fig

date = '3/21/20'
fig = draw_figure(cleaned_df, date)
fig.show()

In [26]:
date_code = date.replace('/', '-')
plotly.offline.plot(fig, filename='cov19-confirmed-'+ date_code + '.html')

'cov19-confirmed-3-21-20.html'